In [10]:
import numpy as np
import random
from copy import deepcopy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

# === CONSTANTES ===
X = 1
O = -1
EMPTY = 0

# === LOGIQUE DU MORPION ===

def check_winner(board):
    wins = [
        [0,1,2], [3,4,5], [6,7,8],
        [0,3,6], [1,4,7], [2,5,8],
        [0,4,8], [2,4,6]
    ]
    for i, j, k in wins:
        if board[i] == board[j] == board[k] != EMPTY:
            return board[i]
    if EMPTY not in board:
        return 0  # Match nul
    return None

def get_available_moves(board):
    return [i for i, val in enumerate(board) if val == EMPTY]

def minimax(board, player):
    winner = check_winner(board)
    if winner is not None:
        return winner * player

    best = -2
    for move in get_available_moves(board):
        board[move] = player
        score = -minimax(board, -player)
        board[move] = EMPTY
        if score > best:
            best = score
    return best

def best_move(board, player):
    best_score = -2
    move_choice = None
    for move in get_available_moves(board):
        board[move] = player
        score = -minimax(board, -player)
        board[move] = EMPTY
        if score > best_score:
            best_score = score
            move_choice = move
    return move_choice

def generate_random_board():
    board = [EMPTY] * 9
    num_moves = random.randint(1, 5)
    current_player = random.choice([X, O])
    for _ in range(num_moves):
        moves = get_available_moves(board)
        if not moves:
            break
        move = random.choice(moves)
        board[move] = current_player
        current_player *= -1
        if check_winner(board) is not None:
            break
    return board, current_player

# === GÉNÉRATION DU DATASET ===

def generate_dataset(n_samples=10000):
    X_data = []
    y_data = []
    for _ in range(n_samples):
        board, player = generate_random_board()
        move = best_move(board, player)
        if move is not None:
            # Perspective du joueur courant
            board_input = [player * val for val in board]
            X_data.append(board_input)
            y_data.append(move)
    return np.array(X_data), to_categorical(y_data, num_classes=9)

# === ENTRAÎNEMENT DU MODÈLE ===

def build_model():
    model = Sequential()
    model.add(Dense(64, input_shape=(9,), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# === PIPELINE COMPLET ===

def train_and_save_model():
    print("📊 Génération des données...")
    X_train, y_train = generate_dataset(20000)

    print("🧠 Construction du modèle...")
    model = build_model()

    print("🚀 Entraînement du modèle...")
    model.fit(X_train, y_train, epochs=30, batch_size=64, validation_split=0.1)

    print("💾 Sauvegarde du modèle...")
    model.save("ia_morpion.h5")
    print("✅ Modèle sauvegardé sous 'ia_morpion.h5'")

# === TESTER LE MODÈLE ===

def load_and_predict(state):
    model = load_model("ia_morpion.h5")
    state = np.array(state).reshape(1, 9)
    prediction = model.predict(state)
    move = np.argmax(prediction)
    return move


In [11]:
# === LANCEMENT ===
if __name__ == "__main__":
    train_and_save_model()

    # Exemple d’utilisation
    test_board = [1, -1, 1,
                  0, -1, 0,
                  0,  0, 1]
    print("🤖 Coup proposé :", load_and_predict(test_board))

📊 Génération des données...
🧠 Construction du modèle...
🚀 Entraînement du modèle...
Epoch 1/30
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3783 - loss: 1.8256 - val_accuracy: 0.6385 - val_loss: 1.1806
Epoch 2/30
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6835 - loss: 1.0296 - val_accuracy: 0.7375 - val_loss: 0.8046
Epoch 3/30
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7720 - loss: 0.7113 - val_accuracy: 0.7990 - val_loss: 0.6278
Epoch 4/30
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8112 - loss: 0.5784 - val_accuracy: 0.8245 - val_loss: 0.5257
Epoch 5/30
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8489 - loss: 0.4848 - val_accuracy: 0.8540 - val_loss: 0.4667
Epoch 6/30
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8684 - loss: 0.4125 - val_accuracy: 0.8655 - val_loss: 0.4136
Epoch 7/30
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8838 - loss: 0.3778 - val_accuracy: 0.8785 - val_loss: 0.3804
Epoch 8/30
282/282 ━

💾 Sauvegarde du modèle...
✅ Modèle sauvegardé sous 'ia_morpion.h5'
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
🤖 Coup proposé : 7
